In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from tqdm import tqdm

In [ ]:
def show_image(images, labele, n = 4):
  plt.figure(figsize = (12, 12))
  for i, image in enumerate(images[:n]):
    plt.subplot(n, n, i + 1)
    image = image / 2 + 0.5
    plt.imshow(image.numpy().transpose((1, 2, 0)).squeeze())
    

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose(
    [ transforms.ToTensor(),
      transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)  )]
)